<a href="https://colab.research.google.com/github/OrJaku/Machine_Learning/blob/master/Boston_housing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.datasets import boston_housing
import numpy as np

(train_data, train_target), (test_data, test_target) = boston_housing.load_data()

In [0]:
mean = train_data.mean(axis=0) #Mean value of train_data
train_data -= mean
# print("Mean", mean)
std = train_data.std(axis=0) #Standard deviation of train_data
train_data /= std

test_data -= mean
test_data /= std


In [0]:
from keras import models
from keras import layers 

def build_model()
    model = models.Sequentail()
    model.add(layers.Dense(64,activation='relu',
                           input_shape=(train_data[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop',
                  loss='mse',
                  metrics=['mae'])
    return model



In [40]:
k = 4
num_val_samples = len(train_data) // k
num_epochs = 100
all_score = []
for i in range(k):
    print("procesing fold #", i)
    val_data = train_data[i * num_val_samples : (i+1) * num_val_samples]
    val_target = train_target[i * num_val_samples : (i+1) * num_val_samples]
print(val_data.shape)
print(val_target.shape)

procesing fold # 0
procesing fold # 1
procesing fold # 2
procesing fold # 3
(101, 13)
(101,)
